In [1]:
import mysql.connector
cnx = mysql.connector.connect(user='', password='',host='')
cursor = cnx.cursor()
USE = 'use DB1;'
cursor.execute(USE)

In [2]:
# la view
dropv = "drop view deadactors;"
createv = ("create view deadactors as "
           "select * "
           "from actors "
           "where datedeath != 3000;")
try:
    cursor.execute(dropv)
except mysql.connector.ProgrammingError as e:
    print(e)
cursor.execute(createv)

In [6]:
# le query
Q = {}

# 1 dato attore vediamo i film (Winona Ryder)
Q['a'] = {}
Q['a']['print'] = 'a. Show all the movies of a specific actor, with the corresponding director and  genres.'
Q['a']['query'] = ("select m.title, m.director, m.year, m.category "
                   "from movies as m, casts as c "
                   "where m.movieid = c.movieid and c.actor = %s;")
Q['a']['output'] = ['title','drector','year','category']
Q['a']['input'] = ['actor']

# 2 top 10 attori che partecipano a piu film con stesso regista
Q['b'] = {}
Q['b']['print'] = 'b. Show the 10 most popular actors, that have worked with the same director'
Q['b']['query'] = ("select count(*), c.actor, m.director "
                   "from movies as m, casts as c "
                   "where m.movieid = c.movieid "
                   "group by m.director, c.actor "
                   "order by count(*) desc "
                   "limit 10;")
Q['b']['output'] = ['count','actor','director']

# 3 attori che hanno lavorato con regista durante la guerra (Hitchcock)
Q['c'] = {}
Q['c']['print'] = ("c. Show actors complete real names which have worked with a specific director during The World War II"
                   ", and that have won an award.")
Q['c']['query'] = ("select A.realsurname, A.realname, m.title, m.director, m.year "
                   "from movies as m, actors as a, casts as c "
                   "where m.movieid = c.movieid and m.year>=1939 and m.year<=1945 and "
                   "c.actor = a.stagename and c.award is not null and m.director = %s;")
Q['c']['output'] = ['surname','name','title','director','year']
Q['c']['input'] = ['director']

# 4 remakes di un certo film('the night of the living dead','romero')
Q['d'] = {}
Q['d']['print'] = ("d. Show title, director, yeare, genres, percentage of similarity of to movies, given the title and the "
                   "director of the original movie")
Q['d']['query'] = ("select m2.title, m2.director, m2.year, m2.category, r.fraction "
                   "from movies as m1, movies as m2, remakes as r "
                   "where m1.movieid = r.priorfilm and m2.movieid = r.movieid "
                   "and m1.title = %s and m1.director = %s;")
Q['d']['output'] = ['title','director','year','category','fraction']
Q['d']['input'] = ['title','director']

# 5 citazioni di un film(Audrey Hepburn)
Q['e'] = {}
Q['e']['print'] = 'e. Show title, director and all the Sayings told by a specific actor(i.e:Audrey Hepburn)'
Q['e']['query'] = ("select m.title, m.director, s.actor, s.col_as, s.col_to, s.saying "
                   "from movies as m, sayings as s "
                   "where s.movieid = m.movieid and s.actor = %s;")
Q['e']['output'] = ['title','director','actor','as','to','saying']
Q['e']['input'] = ['actor']

# 6 film di regista di una certa nazionalita' contenenti nel titolo una certa parola (It,%amore%)(AGGIUNGERE%!!)
Q['f'] = {}
Q['f']['print'] = 'f. Show title and director of all the movies directed by italians which title contains the word amore'
Q['f']['query'] = ("select m.title, m.director "
                   "from movies as m, people as p "
                   "where m.director = p.RefName and background = %s and m.title like %s;")
Q['f']['output'] = ['title','director']
Q['f']['input'] = ['nationality','word']

# 7 film girati in un certo posto (% LA %) con quali studios
Q['g'] = {}
Q['g']['print'] = 'g. Count the studios that produced movies shot in a specific location(i.e: LA,NY)'
Q['g']['query'] = ("select count(*), s.* "
                   "from movies as m, studios as s "
                   "where m.studios = s.name and m.location like %s and m.studios is not null "
                   "group by studios "
                   "order by count(*) desc;")
Q['g']['output'] = ['count','name','fullcompanyname','city','country','year','first','founder','successor','notes']
Q['g']['input'] = ['location']

# 8 Nicolas Cage
Q['h'] = {}
Q['h']['print'] = ("h. Show title, Director and Year of the a movie that has been direceted by 'John Woo' and  has been "
                   "interpreted by an actor whose real surname is Coppola, its a male and was born after 1940 ")
Q['h']['query'] = ("select m.Title, m.Director, m.Year "
                   "from movies as m, casts as c "
                   "where m.Director = 'Woo'  and c.Actor = "
                   "(select StageName "
                   "from actors "
                   "where RealSurname = 'Coppola' and Gender = 'M' and DateBirth > 1940);")
Q['h']['output'] = ['title','director','year']

# 9 contare quanti film per ogni regista italiano che hanno vinto dei premi e sono nati durante il regime fascista
Q['i'] = {}
Q['i']['print'] = 'i. Show all the film of italain directores that have won an award and they were born during fascist regime'
Q['i']['query'] =  ("from people as p , movies as m "
                   "where p.RefName = m.Director and p.DateBirth >= 1921 and p.DateBirth <= 1941 "
                   "and p.Background = 'IT' and m.Awards is not null "
                   "group by m.Director "
                   "order by count(*) desc;")
Q['i']['output'] = ['count','director']

# 10 restituire tutti i film fatti a colori poi ordinare per anno e prendere il minimo
Q['l'] = {}
Q['l']['print'] = ("l. Show the title, year and color process of the films that have been pioneers using color techniques"
                   "(in the same year for the first time )")
Q['l']['query'] = ("select m.Title, m.Year, cp.FullName, cp.Description "
                    "from movies as m, colorprocess as cp "
                    "where m.ColorProcess = 'col' and cp.Code = m.ColorProcess and m.Year = ( "
                    "select min(m.Year) "
                    "from movies as m "
                    "where m.ColorProcess = 'col');")
Q['l']['output'] = ['title','year','colorprocess','description']

# 11
Q['m'] = {}
Q['m']['print'] = ("m. Show the nationalities and their role and the number of movies and names of people that are both a "
                   "writer and a V.art")
Q['m']['query'] = ("select count(*), p.RefName, p.Codes, p.Background "
                   "from people as p, movies as m "
                   "where m.Director = p.RefName and p.codes like '%V%' and p.codes like '%W%' "
                   "group by m.Director; ")
Q['m']['output'] = ['count','name','codes','background']

# 12
Q['n'] = {}
Q['n']['print'] = ("n. Show the title, the director, the year, the category and the number of remakes of movies of a "
                   "given genres")
Q['n']['query'] = ("select count(*), m.title, m.Director, m.Year, m.Category "
                   "from movies as m, remakes as r "
                   "where m.MovieId = r.PriorFilm and m.Category= %s "
                   "group by r.PriorFilm "
                   "having count(*)>1 "
                   "order by count(*) desc;")
Q['n']['output'] = ['count','title','director','year','category']
Q['n']['input'] = ['genres']


# 13   10 attori morti più prolifici usando views
Q['o'] = {}
Q['o']['print'] = 'o. Show the top 10 of the most popular dead actors using views'
Q['o']['query'] = ("select count(*), c.actor, da.datedeath "
                    "from hw1.casts as c, hw1.deadactors as da "
                    "where c.actor = da.stagename "
                    "group by c.actor "
                    "order by count(*) desc "
                    "limit 10")
Q['o']['output'] = ['count','actor','datedeath']

In [7]:
# dati preimpostati
data = {}
data['a'] = ['Winona Ryder']
data['c'] = ['Hitchcock']
data['d'] = ['the night of the living dead','romero']
data['e'] = ['Audrey Hepburn']
data['f'] = ['It','%amore%']
data['g'] = ['% LA %']
data['n'] = ['Horr']


# se si preme enter si interrompe
# se si preme qualcosa di diverso da p quando chiede i parameters usa i parameters preimpostati

for k in sorted(Q.keys()):
    print(Q[k]['print'])
stop = False
while(stop==False):
    q = input('query: ')
    if q == '':
        stop = True
        continue
    print(Q[q]['print'])
    print('parameters:')
    if 'input' in Q[q].keys():
        for p in range(len(Q[q]['input'])):
            print(Q[q]['input'][p]+':',data[q][p])
        param = input('to change the parameters press p')
        if param == 'p':
            for p in range(len(Q[q]['input'])):
                data[q][p] = input(Q[q]['input'][p]+': ')
    print('\n')
    # finalmente eseguiamo la query
    if q in data.keys():
        cursor.execute(Q[q]['query'],data[q])
    else:
        cursor.execute(Q[q]['query'])
    for out in cursor:
        for c in range(len(out)):
            print(Q[q]['output'][c]+':',out[c])
        print('\n')

a. Show all the movies of a specific actor, with the corresponding director and  genres.
b. Show the 10 most popular actors, that have worked with the same director
c. Show actors complete real names which have worked with a specific director during The World War II, and that have won an award.
d. Show title, director, yeare, genres, percentage of similarity of to movies, given the title and the director of the original movie
e. Show title, director and all the Sayings told by a specific actor(i.e:Audrey Hepburn)
f. Show title and director of all the movies directed by italians which title contains the word amore
g. Count the studios that produced movies shot in a specific location(i.e: LA,NY)
h. Show title, Director and Year of the a movie that has been direceted by 'John Woo' and  has been interpreted by an actor whose real surname is Coppola, its a male and was born after 1940 
i. Show all the film of italain directores that have won an award and they were born during fascist regime